In [1]:
import plotly.express as px
import re
import plotly.io as pio
# pio.renderers.default = "browser"
pio.renderers.default = "notebook_connected"
from collections import OrderedDict
import argparse
from scipy.stats import gmean
import numpy as np
import plotly.graph_objects as go
import plotly.figure_factory as ff
import os

In [2]:
from util import get_list, to_matrix, pattern_map, prune_map, bcolors
from util import spec17_working_set_map, spec17_bench_map

# Configure result directory below
Note: only the last cell will be analysed!

## parsec
if you are using parsec configure cell below and comment spec2017 cell

In [3]:
# time_str = '20220728160200'
# bench = ['blackscholes',\
#          'bodytrack', \
#          'canneal', \
#          'dedup', \
#          'fluidanimate', \
#          'raytrace' , \
#          'swaptions', \
#         ]
# sim_str = 'simlarge'
# config = [6]
# core = 64
# outdir_hist = [f'{time_str}/{b}_{sim_str}_{core}_{c}/stats.txt' for b in bench for c in config ]
# id_str = '_6'
# outdir_hist

## spec2017

In [4]:
# time_str = '20220806185100'
# set__ = [3]
# sync = [2]
# core = 16
# outdir_hist = [f'{time_str}/set{set_}_sync{sync_}_c{core}_0/stats.txt' for set_ in set__ for sync_ in sync ]
# id_str = '_0'
# bench = [f'set{set_}_s{sync_}' for set_ in set__ for sync_ in sync]
# outdir_hist

In [5]:
# time_str = '20220808193300'
# set__ = [2]
# set_encoding = [spec17_bench_map[n] for n in set__]
# sync = [0, 1, 4, 8, 16]
# core = 4
# outdir_hist = [f'{time_str}/set{set_}_sync{sync_}_c{core}_0/stats.txt' for set_ in set__ for sync_ in sync ]
# id_str = '_0'
# outdir_hist
# bench = [f'set{set_}_s{sync_}:{set_enc}' for set_ in set__ for sync_ in sync for set_enc in set_encoding]

thiss sett  is rrun by seeeing onnly 1 workend

In [6]:
time_str = '20220810131400'
set__ = [10]
set_encoding = [spec17_bench_map[n] for n in set__]
sync = [4 if s != 11 else 6 for s in set__]
util = [25,50,100]
core = 16
outdir_hist = [f'{time_str}/set{set_}_sync{sync_}_c{core}_u{util_}_0/stats.txt' for set_,sync_ in zip(set__,sync) for util_ in util]
id_str = '_0'

bench = [f'set{set_}_s{sync_}_u{util_}' for (set_,sync_) in zip(set__,sync) for util_ in util]
bench_detail = [f'{set_enc}' for set_enc in set_encoding for b in bench]
outdir_hist

['20220810131400/set10_sync4_c16_u25_0/stats.txt',
 '20220810131400/set10_sync4_c16_u50_0/stats.txt',
 '20220810131400/set10_sync4_c16_u100_0/stats.txt']

tthis seet is rrun by  seeeing  all workendds

In [7]:
# time_str = '20220817091200'
# set__ = [10]
# set_encoding = [spec17_bench_map[n] for n in set__]
# sync = [2]
# util = [100]
# core = 16
# outdir_hist = [f'{time_str}/c{core}_set{set_}_sync{sync_}_u{util_}_0/stats.txt' for set_,sync_ in zip(set__,sync) for util_ in util]
# id_str = '_0'

# bench = [f'set{set_}_s{sync_}_u{util_}' for (set_,sync_) in zip(set__,sync) for util_ in util]
# bench_detail = [f'{set_enc}' for set_enc in set_encoding for b in bench]
# outdir_hist

In [8]:
bench

['set10_s4_u25', 'set10_s4_u50', 'set10_s4_u100']

In [9]:
bench_detail

["['povray', 'cam4', 'nab', 'mcf', 'namd', 'omnetpp', 'lbm', 'fotonik3d']",
 "['povray', 'cam4', 'nab', 'mcf', 'namd', 'omnetpp', 'lbm', 'fotonik3d']",
 "['povray', 'cam4', 'nab', 'mcf', 'namd', 'omnetpp', 'lbm', 'fotonik3d']"]

### run to check if all required files exist

In [10]:
for path_to_file in outdir_hist:
    if os.path.exists(path_to_file)==False or \
        os.stat(path_to_file).st_size == 0: 
        print(bcolors.FAIL + f'File {path_to_file} is empty or does not exist'+ bcolors.ENDC)

### run to check if any hostSeconds too short to be valid

In [11]:
metric = 'hostseconds'
hs = [get_list(o, metric) for o in outdir_hist if id_str in o]
for i in range(len(hs)):
    s = hs[i]
    assert len(s) == 1
    if s[0] < 10: print(bcolors.WARNING + f'Suspicious {bench[i]} with really short run time of {s[0]}s' + bcolors.ENDC)
hs        

[[15838.69], [25283.24], [52282.23]]

In [12]:
mesh_dim = int(np.sqrt(core))
mesh_dim

4

In [13]:
mid = float(100/(mesh_dim*mesh_dim))
mid

6.25

In [14]:
metric = 'l1_ua'
l1_ua_bins = [get_list(o, metric) for o in outdir_hist if id_str in o]
# post-processing
# print(l1_ua_bins)
l1_bin_size = l1_ua_bins[0][1][0]
l1_num_bins = int(3200 / l1_bin_size)
l1_ua_bins = [[(int(tuple_[0]/l1_bin_size), tuple_[1]) for tuple_ in arr] for arr in l1_ua_bins]
l1_ua_bins = [to_matrix(arr, l1_num_bins) for arr in l1_ua_bins]

metric = 'l2_ua'
l2_ua_bins = [get_list(o, metric) for o in outdir_hist if id_str in o]
# post-processing
l2_bin_size = l2_ua_bins[0][1][0]
l2_num_bins = int(3200 / l2_bin_size)
l2_ua_bins = [[(int(tuple_[0]/l2_bin_size), tuple_[1]) for tuple_ in arr] for arr in l2_ua_bins]
l2_ua_bins = [to_matrix(arr, l2_num_bins) for arr in l2_ua_bins]

In [15]:
l1_bin_cutoff = 1
l1_theta = l1_bin_cutoff * l1_bin_size
l1_hot = [[[tuple_[1] for tuple_ in bins] for bins in percore] for percore in l1_ua_bins]
l1_hot = [[sum(bins[l1_bin_cutoff:l1_num_bins-1]) for bins in percore] for percore in l1_hot]

l1_uniqueaccess = [[[tuple_[1] for tuple_ in bins] for bins in percore] for percore in l1_ua_bins]
l1_uniqueaccess = [[sum(bins) for bins in percore] for percore in l1_uniqueaccess]

l1_hot_all_core = [sum(arr) for arr in l1_hot]
l1_uniqueaccess_all_core = [sum(arr) for arr in l1_uniqueaccess]
print('l1:', l1_hot_all_core,l1_uniqueaccess_all_core)

l1_hot_norm = [[float(percore/sum_)*100 for percore in arr] for arr, sum_ in zip(l1_hot,l1_hot_all_core)]
l1_uniqueaccess_norm = [[float(percore/sum_)*100 for percore in arr] for arr, sum_ in \
                        zip(l1_uniqueaccess,l1_uniqueaccess_all_core)]
# l1_std.append()

l2_bin_cutoff = 1
l2_theta = l2_bin_cutoff * l2_bin_size
l2_hot = [[[tuple_[1] for tuple_ in bins] for bins in percore] for percore in l2_ua_bins]
l2_hot = [[sum(bins[l2_bin_cutoff:l2_num_bins-1]) for bins in percore] for percore in l2_hot]

l2_uniqueaccess = [[[tuple_[1] for tuple_ in bins] for bins in percore] for percore in l2_ua_bins]
l2_uniqueaccess = [[sum(bins) for bins in percore] for percore in l2_uniqueaccess]

l2_hot_all_core = [sum(arr) for arr in l2_hot]
l2_uniqueaccess_all_core = [sum(arr) for arr in l2_uniqueaccess]
print('l2:', l2_hot_all_core,l2_uniqueaccess_all_core)

l2_hot_norm = [[float(percore/sum_)*100 for percore in arr] for arr, sum_ in zip(l2_hot,l2_hot_all_core)]
l2_uniqueaccess_norm = [[float(percore/sum_)*100 for percore in arr] for arr, sum_ in \
                        zip(l2_uniqueaccess,l2_uniqueaccess_all_core)]

l1: [707593, 4130118, 8142297] [6700467, 21375866, 42634348]
l2: [333423, 1015505, 1970606] [6687580, 21365126, 42618371]


# plot hot line distribution

In [16]:
l1_std_lineutil = []
l2_std_lineutil = []
for l1_arr, l2_arr, l1_anno_arr, l2_anno_arr, \
    name, name_d, l1_hot_total, l2_hot_total \
    in zip(l1_hot_norm, l2_hot_norm, l1_hot_norm, l2_hot_norm, \
        bench, bench_detail, l1_hot_all_core, l2_hot_all_core):
    
    l1_matrix = to_matrix(l1_arr, mesh_dim)
    l2_matrix = to_matrix(l2_arr, mesh_dim)
    l1_anno_matrix = to_matrix(l1_anno_arr, mesh_dim)
    l2_anno_matrix = to_matrix(l2_anno_arr, mesh_dim)
    
    l1_anno_matrix = [ [round(elem, 2) for elem in my_list] for my_list in l1_anno_matrix ]
    l2_anno_matrix = [ [round(elem, 2) for elem in my_list] for my_list in l2_anno_matrix ]

    
    # l1 heatmap
    l1_std_ = np.std(l1_arr)
    l1_std_lineutil.append(l1_std_)
    
    fig = go.Figure(data=go.Heatmap(
                    z=l1_matrix,
                    colorscale='RdBu_r',
                    zauto=True,
                    zmid=mid,
                    text=l1_anno_matrix,
                    texttemplate="%{text}",
                    textfont={"size":15}))
    fig.update_layout(
        title=f"{name}: std={round(l1_std_, 4)}, <br>" \
            f"{l1_hot_total} total l1 hot (access>={l1_theta}) lines among all banks<br><sup>{name_d}</sup>",
    )
    fig.update_xaxes(visible=False)
    fig.update_yaxes(visible=False)
    fig.layout.height = 500
    fig.layout.width = 500
    
    fig.show()
    
    # l2 heatmap
    l2_std_ = np.std(l2_arr)
    l2_std_lineutil.append(l2_std_)
    
    fig = go.Figure(data=go.Heatmap(
                    z=l2_matrix,
                    colorscale='Tropic',
                    zauto=True,
                    zmid=mid,
                    text=l2_anno_matrix,
                    texttemplate="%{text}",
                    textfont={"size":15}))
    fig.update_layout(
        title=f"{name}: std={round(l2_std_, 4)}, <br>" \
            f"{l2_hot_total} total hot (access>={l2_theta}) lines among all banks<br><sup>{name_d}</sup>",
    )
    fig.update_xaxes(visible=False)
    fig.update_yaxes(visible=False)
    fig.layout.height = 500
    fig.layout.width = 500
    
    fig.show()
#     break

# plot unique line distribution

In [17]:
l1_std_lineutil = []
l2_std_lineutil = []
for l1_arr, l2_arr, l1_anno_arr, l2_anno_arr, \
    name, name_d, l1_uniqueaccess_total, l2_uniqueaccess_total \
    in zip(l1_uniqueaccess_norm, l2_uniqueaccess_norm, l1_uniqueaccess_norm, l2_uniqueaccess_norm, \
        bench, bench_detail, l1_uniqueaccess_all_core, l2_uniqueaccess_all_core):
    
    # make square and process annotation
    l1_matrix = to_matrix(l1_arr, mesh_dim)
    l2_matrix = to_matrix(l2_arr, mesh_dim)
    
    l1_anno_matrix = to_matrix(l1_anno_arr, mesh_dim)
    l2_anno_matrix = to_matrix(l2_anno_arr, mesh_dim)
    
    l1_anno_matrix = [ [round(elem, 2) for elem in my_list] for my_list in l1_anno_matrix ]
    l2_anno_matrix = [ [round(elem, 2) for elem in my_list] for my_list in l2_anno_matrix ]

    
    # l1 heatmap
    l1_std_ = np.std(l1_arr)
    l1_std_lineutil.append(l1_std_)
    
    fig = go.Figure(data=go.Heatmap(
                    z=l1_matrix,
                    colorscale='RdBu_r',
                    zauto=True,
                    zmid=mid,
                    text=l1_anno_matrix,
                    texttemplate="%{text}",
                    textfont={"size":15}))
    fig.update_layout(
        title=f"{name}: std={round(l1_std_, 4)}, <br>" \
            f"{l1_uniqueaccess_total} total unique lines among all banks<br><sup>{name_d}</sup>",
    )
    fig.update_xaxes(visible=False)
    fig.update_yaxes(visible=False)
    fig.layout.height = 500
    fig.layout.width = 500
    fig.show()
    
    # l2 heatmap
    l2_std_ = np.std(l2_arr)
    l2_std_lineutil.append(l2_std_)
    
    fig = go.Figure(data=go.Heatmap(
                    z=l2_matrix,
                    colorscale='Tropic',
                    zauto=True,
                    zmid=mid,
                    text=l2_anno_matrix,
                    texttemplate="%{text}",
                    textfont={"size":15}))
    fig.update_layout(
        title=f"{name}: std={round(l2_std_, 4)}, <br>" \
            f"{l2_uniqueaccess_total} total unique lines among all banks<br><sup>{name_d}</sup>",
    )
    fig.update_xaxes(visible=False)
    fig.update_yaxes(visible=False)
    fig.layout.height = 500
    fig.layout.width = 500
    fig.show()
#     break

# Conclusions
## set9: heavy
- figured out core mapping

| coremapping |  |  |  |  |
| --- | --- | --- | --- | --- |
| | 6 | 7 | 8 | 9 
| | 2 | 3 | 4 | 5 
| | 12 | 13 | 14 | 15 
| | 0 | 1 | 10 | 11 

## set10: hetro
- L2 unique lines distribution is pretty unifom with very low standard deviation.
- L1 unique lines distribution aligns with program working set. i.e., the larger the working set, the more unique lines accessed by **the last level private cache**, (even though this is filtered by lower level caches already, this trend still persists.)

## set11: light
- util=100 runs for too short, bogus

## set12: medium